In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = "https://fbref.com/pt/comps/9/cronograma/Premier-League-Resultados-e-Calendarios"
# Faz a requisição para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Função para extrair a tabela da página
def extrair_tabela(rows):
    headers = []
    data = []
    
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(headers) == 0:  # A primeira linha será o cabeçalho
            headers = [col.get_text(strip=True) for col in cols]
        else:  # Demais linhas são dados
            data.append([col.get_text(strip=True) for col in cols])
    
    # Cria o DataFrame
    return pd.DataFrame(data, columns=headers)

# Encontra o container da tabela
table_container = soup.find('div', {'class': 'table_container'})
rows = table_container.find_all('tr')

# Extrai a tabela
tabela = extrair_tabela(rows)

# Encontra a coluna "Relatório da Partida"
links_relatorios = []
for row in rows:
    a_tag = row.find('a', text='Relatório da Partida')
    if a_tag:
        href = a_tag.get('href')
        if href and href != "#":  # Ignora links vazios ou "#" como marcador
            links_relatorios.append(href)
        else:
            links_relatorios.append(None)
    else:
        links_relatorios.append(None)

# Remove linhas onde a coluna "Relatório da Partida" é "Confronto"
tabela = tabela[~tabela['Relatório da Partida'].str.contains("Confronto", na=False)]

# Adiciona a coluna de links na tabela
tabela['Link Relatório'] = links_relatorios[:len(tabela)]  # Ajusta o tamanho da lista para o tamanho da tabela

# Exibe a tabela final com os links
print("\nTabela com links:")
print(tabela.head())

# Se precisar dos links, aqui está a lista filtrada
print("\nLinks 'Relatório da Partida':")
print(tabela['Link Relatório'].dropna().tolist())



Tabela com links:
  Sem  Dia        Data Horário          Em casa   xG Resultado   xG  \
0   1  sex  2024-08-16   20:00   Manchester Utd  2.4       1–0  0.4   
1   1  sáb  2024-08-17   12:30     Ipswich Town  0.5       0–2  2.6   
2   1  sáb  2024-08-17   15:00    Newcastle Utd  0.3       1–0  1.8   
3   1  sáb  2024-08-17   15:00  Nott'ham Forest  1.3       1–1  1.2   
4   1  sáb  2024-08-17   15:00          Everton  0.5       0–3  1.4   

     Visitante Público                 Local         Árbitro  \
0       Fulham  73.297          Old Trafford    Robert Jones   
1    Liverpool  30.014  Portman Road Stadium    Tim Robinson   
2  Southampton  52.196        St James' Park    Craig Pawson   
3  Bournemouth  29.763       The City Ground  Michael Oliver   
4     Brighton  39.217         Goodison Park    Simon Hooper   

   Relatório da Partida Notas  \
0  Relatório da Partida         
1  Relatório da Partida         
2  Relatório da Partida         
3  Relatório da Partida         
4  R

C:\Users\eduar\AppData\Local\Temp\ipykernel_32028\3808233629.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = row.find('a', text='Relatório da Partida')


In [2]:
# Desloca a coluna "Link Relatório" uma linha para cima
tabela['Link Relatório'] = tabela['Link Relatório'].shift(-1)

# Preenche a última linha da coluna "Link Relatório" com NaN (caso não esteja automaticamente)
tabela.loc[tabela.index[-1], 'Link Relatório'] = None


In [3]:
# Remove linhas onde "Link Relatório" é None
tabela = tabela[tabela['Link Relatório'].notna()]

In [4]:
tabela

,Sem,Dia,Data,Horário,Em casa,xG,Resultado,xG,Visitante,Público,Local,Árbitro,Relatório da Partida,Notas,Link Relatório
0,1,sex,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,73.297,Old Trafford,Robert Jones,Relatório da Partida,,/pt/partidas/cc5b4244/Manchester-United-Fulham...
1,1,sáb,2024-08-17,12:30,Ipswich Town,0.5,0–2,2.6,Liverpool,30.014,Portman Road Stadium,Tim Robinson,Relatório da Partida,,/pt/partidas/a1d0d529/Ipswich-Town-Liverpool-2...
2,1,sáb,2024-08-17,15:00,Newcastle Utd,0.3,1–0,1.8,Southampton,52.196,St James' Park,Craig Pawson,Relatório da Partida,,/pt/partidas/34557647/Newcastle-United-Southam...
3,1,sáb,2024-08-17,15:00,Nott'ham Forest,1.3,1–1,1.2,Bournemouth,29.763,The City Ground,Michael Oliver,Relatório da Partida,,/pt/partidas/4efc72e4/Nottingham-Forest-Bourne...
4,1,sáb,2024-08-17,15:00,Everton,0.5,0–3,1.4,Brighton,39.217,Goodison Park,Simon Hooper,Relatório da Partida,,/pt/partidas/71618ace/Everton-Brighton-and-Hov...
5,1,sáb,2024-08-17,15:00,Arsenal,1.2,2–0,0.5,Wolves,60.261,Emirates Stadium,Jarred Gillett,Relatório da Partida,,/pt/partidas/c0e3342a/Arsenal-Wolverhampton-Wa...
6,1,sáb,2024-08-17,17:30,West Ham,2.3,1–2,2.0,Aston Villa,62.463,London Stadium,Tony Harrington,Relatório da Partida,,/pt/partidas/eac7c00b/West-Ham-United-Aston-Vi...
7,1,dom,2024-08-18,14:00,Brentford,1.6,2–1,1.2,Crystal Palace,16.988,Gtech Community Stadium,Samuel Barrott,Relatório da Partida,,/pt/partidas/b63822b9/Brentford-Crystal-Palace...
8,1,dom,2024-08-18,16:30,Chelsea,1.0,0–2,0.8,Manchester City,39.818,Stamford Bridge,Anthony Taylor,Relatório da Partida,,/pt/partidas/67a0c715/Chelsea-Manchester-City-...
9,1,seg,2024-08-19,20:00,Leicester City,1.0,1–1,1.2,Tottenham,31.977,King Power Stadium,Chris Kavanagh,Relatório da Partida,,/pt/partidas/62eea1d6/Leicester-City-Tottenham...


In [5]:
tabela.to_csv('C://Users//eduar//Desktop//DEV//Football//PremierLeague//links.csv', index= False)